In [1]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
print ('Ran')

Ran


In [2]:
prev_app = pd.read_csv('D:/Project_Data/previous_application.csv')

In [3]:
prev_app.shape

(1670214, 37)

prev_app=prev_app.iloc[0:1048575,:]

prev_app.shape

In [4]:
flag_col=[col for col in prev_app.columns if 'FLAG_' in col]

In [5]:
flag_col

['FLAG_LAST_APPL_PER_CONTRACT',
 'NFLAG_LAST_APPL_IN_DAY',
 'NFLAG_INSURED_ON_APPROVAL']

In [6]:
cat_features = [f for f in prev_app.columns if prev_app[f].dtype == 'object' and f not in flag_col]

In [7]:
cat_features

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION']

In [8]:
num_features = [f for f in prev_app.columns if prev_app[f].dtype != 'object' and f not in flag_col]

In [9]:
num_features

['SK_ID_PREV',
 'SK_ID_CURR',
 'AMT_ANNUITY',
 'AMT_APPLICATION',
 'AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
 'AMT_GOODS_PRICE',
 'HOUR_APPR_PROCESS_START',
 'RATE_DOWN_PAYMENT',
 'RATE_INTEREST_PRIMARY',
 'RATE_INTEREST_PRIVILEGED',
 'DAYS_DECISION',
 'SELLERPLACE_AREA',
 'CNT_PAYMENT',
 'DAYS_FIRST_DRAWING',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION']

In [10]:
df = prev_app[num_features]

In [11]:
df.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,0.0,0.182832,0.867336,-73,35,12.0,365243.0,-42.0,300.0,-42.0,-37.0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,NaN,NaN,NaN,-164,-1,36.0,365243.0,-134.0,916.0,365243.0,365243.0
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,11,NaN,NaN,NaN,-301,-1,12.0,365243.0,-271.0,59.0,365243.0,365243.0
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,7,NaN,NaN,NaN,-512,-1,12.0,365243.0,-482.0,-152.0,-182.0,-177.0
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,9,NaN,NaN,NaN,-781,-1,24.0,NaN,NaN,NaN,NaN,NaN


In [12]:
def agg_numeric(df, group_var):

    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = df.groupby(group_var).agg('mean').reset_index()

    return agg

In [13]:
num_group_df = agg_numeric(df.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR')

In [14]:
num_group_df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
0,100001,3951.000,24835.50,23787.00,2520.0,24835.5,13.000000,0.104326,NaN,NaN,-1740.0,23.0,8.0,365243.0,-1709.000000,-1499.000000,-1619.000000,-1612.000000
1,100002,9251.775,179055.00,179055.00,0.0,179055.0,9.000000,0.000000,NaN,NaN,-606.0,500.0,24.0,365243.0,-565.000000,125.000000,-25.000000,-17.000000
2,100003,56553.990,435436.50,484191.00,3442.5,435436.5,14.666667,0.050030,NaN,NaN,-1305.0,533.0,10.0,365243.0,-1274.333333,-1004.333333,-1054.333333,-1047.333333
3,100004,5357.250,24282.00,20106.00,4860.0,24282.0,5.000000,0.212008,NaN,NaN,-815.0,30.0,4.0,365243.0,-784.000000,-694.000000,-724.000000,-714.000000
4,100005,4813.200,22308.75,20076.75,4464.0,44617.5,10.500000,0.108964,NaN,NaN,-536.0,18.0,12.0,365243.0,-706.000000,-376.000000,-466.000000,-460.000000


In [15]:
num_group_df.shape

(338857, 18)

In [16]:
categorical = pd.get_dummies(prev_app[cat_features])
categorical['SK_ID_CURR'] = prev_app['SK_ID_CURR']
categorical.head()

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,...,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,SK_ID_CURR
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,271877
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,108129
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,122040
3,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,176158
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,202054


In [17]:
cat_group_df = categorical.groupby('SK_ID_CURR').agg('sum').reset_index()
cat_group_df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,...,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,100001,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,100002,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,100003,1,2,0,0,1,0,1,1,0,...,1,0,1,0,1,0,0,0,0,0
3,100004,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,100005,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [18]:
cat_group_df.shape

(338857, 142)

In [19]:
flag = prev_app[flag_col]

In [20]:
flag['SK_ID_CURR'] = prev_app['SK_ID_CURR']

C:\Users\dbda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
flag = pd.get_dummies(flag)

In [22]:
flag.head()

,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL,SK_ID_CURR,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y
0,1,0.0,271877,0,1
1,1,1.0,108129,0,1
2,1,1.0,122040,0,1
3,1,1.0,176158,0,1
4,1,NaN,202054,0,1


In [23]:
flag_group_df = flag.groupby('SK_ID_CURR').agg('sum').reset_index()

In [24]:
flag_group_df[flag_group_df['FLAG_LAST_APPL_PER_CONTRACT_N']==1]

,SK_ID_CURR,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y
55,100059,17,0.0,1,17
86,100091,5,0.0,1,5
106,100113,4,1.0,1,3
126,100135,11,3.0,1,11
225,100239,13,0.0,1,13
361,100382,6,2.0,1,6
414,100437,10,2.0,1,10
457,100485,8,0.0,1,7
459,100487,8,2.0,1,8
657,100695,10,1.0,1,10


In [25]:
flag_group_df.shape

(338857, 5)

In [26]:
prev_app = num_group_df.merge(cat_group_df,on='SK_ID_CURR')

In [27]:
prev_app.shape

(338857, 159)

In [28]:
prev_app = prev_app.merge(flag_group_df,on='SK_ID_CURR')

In [29]:
prev_app.shape

(338857, 163)

In [30]:
prev_app.to_csv('C:/Users/dbda/Desktop/Work/Preprocessing_mean/prev_app_aggregate.csv',index=False)